# Stock reccomender
Steve Smith
Jan 2025

The purpose of this is to build an AI agent that will reccomend stocks, 

# Fetch data

In [8]:
from tensortrade.data.cdd import CryptoDataDownload

import numpy as np
import pandas as pd
pd.options.mode.use_inf_as_na = True

def prepare_data(df):
    df['volume'] = np.int64(df['volume'])
    df['date'] = pd.to_datetime(df['date'])
    df.sort_values(by='date', ascending=True, inplace=True)
    df.reset_index(drop=True, inplace=True)
    df['date'] = df['date'].dt.strftime('%Y-%m-%d %I:%M %p')
    return df

#   CryptoDataDownload  Parameters
#         ----------
#         exchange_name : str
#             The name of the exchange.
#         base_symbol : str
#             The base symbol fo the cryptocurrency pair.
#         quote_symbol : str
#             The quote symbol fo the cryptocurrency pair.
#         timeframe : {"d", "h", "m"}
#             The timeframe to collect data from.
#         include_all_volumes : bool, optional
#             Whether or not to include both base and quote volume.

#         Returns
#         -------
#         `pd.DataFrame`
#             A open, high, low, close and volume for the specified exchange and
#             cryptocurrency pair.
#         """

def fetch_data(exchange_name="Bitfinex",quote_symbol="BTC",timeframe="1h"):
    # simulated data for now?
    cdd = CryptoDataDownload()
    bitfinex_data = cdd.fetch(exchange_name, "USD", quote_symbol, timeframe)
    bitfinex_data = bitfinex_data[['date', 'open', 'high', 'low', 'close', 'volume']]
    bitfinex_data = prepare_data(bitfinex_data)
    return bitfinex_data

def load_csv(filename):
    df = pd.read_csv('data/' + filename, skiprows=1)
    df.drop(columns=['symbol', 'volume_btc'], inplace=True)

    # Fix timestamp from "2019-10-17 09-AM" to "2019-10-17 09-00-00 AM"
    df['date'] = df['date'].str[:14] + '00-00 ' + df['date'].str[-2:]

    return prepare_data(df)

#TODO : play with data size parameters such as timefrome (increases number of rows)
#TODO: make sure parameters confrom to the acceptable set
market_data = fetch_data(timeframe="1h")
market_data

/var/folders/vv/lg2w1mmx79b93kh1fxk_v5p00000gn/T/ipykernel_13003/3395647669.py:5: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  pd.options.mode.use_inf_as_na = True


,date,open,high,low,close,volume
0,2018-05-15 06:00 AM,8723.8,8793.0,8714.9,8739.0,8988053
1,2018-05-15 07:00 AM,8739.0,8754.8,8719.3,8743.0,2288904
2,2018-05-15 08:00 AM,8743.0,8743.1,8653.2,8723.7,8891773
3,2018-05-15 09:00 AM,8723.7,8737.8,8701.2,8708.1,2054868
4,2018-05-15 10:00 AM,8708.1,8855.7,8695.8,8784.4,17309722
...,...,...,...,...,...,...
58178,2025-01-02 09:00 PM,97378.0,97394.0,97128.0,97262.0,174426
58179,2025-01-02 10:00 PM,97261.0,97261.0,96777.0,97000.0,1611671
58180,2025-01-02 11:00 PM,96984.0,97106.0,96871.0,97035.0,490811
58181,2025-01-03 12:00 AM,97035.0,97101.0,96872.0,96911.0,2174059
